<a href="https://colab.research.google.com/github/gusya-soc/notebook_collection/blob/main/techc_livedoor_news_corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
apt install mecab libmecab-dev mecab-ipadic-utf8 swig
pip install gensim mecab-python3==0.7
wget http://www.rondhuit.com/download/ldcc-20140209.tar.gz
tar zfx ldcc-20140209.tar.gz

Reading package lists...
Building dependency tree...
Reading state information...
libmecab-dev is already the newest version (0.996-5).
mecab is already the newest version (0.996-5).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-1).
swig is already the newest version (3.0.12-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.




--2020-01-20 01:39:50--  http://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.rondhuit.com/download/ldcc-20140209.tar.gz [following]
--2020-01-20 01:39:51--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz.1’

     0K .......... .......... .......... .......... ..........  0% 74.2K 1m56s
    50K .......... .......... .......... .......... ..........  1%  222K 77s
   100K .......... .......... .......... .......... ..........  1%  111K 76s
   150K .......... .......... .......... .......... ..........  2%  222K 66s
   200K .......... .......... ....

In [ ]:
# ! ls text/sports-watch
! cat text/sports-watch/sports-watch-5250542.txt

http://news.livedoor.com/article/detail/5250542/
2011-01-05T19:30:00+0900
【Sports Watch】香川、ドイツで活躍する要因語る
4日放送、テレビ朝日「報道ステーション」では、ドイツ・ブンデスリーガのドルトムントに所属する香川真司が出演。172cmと小柄で、体格では後れをとりながらも、ドイツの地でゴールを量産する、その要因を語った。

8月22日、ドイツリーグの開幕戦（vsレバークーゼン）で開始9分、最初のシュートをねらった場面を挙げた香川は、そのファーストタッチで相手DFを一瞬にして抜き去ったことに触れると、「自分の中ではイメージ通り、ファーストタッチで相手を抜けました。ファーストタッチでいかに前を向いて、どこにボールを置くか。常に動きながら、ボールを蹴るように意識していますね。いかにスペースにボールを運んでトップスピードになれるか。止まらず、入りこんでいく姿勢が今ゴールに繋がっているのかなと思います」と明かした。

また、今月行われるAFCアジアカップについては、“仕掛ける”とフリップに書き込み、香川は「常にゴールに向かって仕掛ける。自分の真価が問われるので楽しみ」と意気込むのだった。


In [ ]:
import os
DATA_DIR_PATH = "./text"
def get_dir_list():
    categories = [ ] # ここにフォルダ名一覧を入れる
    for dirname in os.listdir(DATA_DIR_PATH):
        path = os.path.join(DATA_DIR_PATH, dirname)
        if os.path.isdir(path):
            categories.append(dirname)
    return categories

def get_contents():
    categories = get_dir_list()
    results = { }
    for dirname in categories:
        path = os.path.join(DATA_DIR_PATH, dirname)
        for filename in os.listdir(path):
            filepath = os.path.join(path, filename)
            with open(filepath, encoding="utf-8") as f:
                lines = f.readlines() # 全行読み込む
                content = "".join(lines[2:]) #全ての行を連結
                results[filename] = content
    return results

import MeCab
mecab = MeCab.Tagger("mecabrc")
mecab.parseToNode('')
def tokenize(text):
    node = mecab.parseToNode(text)
    while node:
        pos = node.feature.split(",")[0]
        if pos in ["名詞", "形容詞", "動詞", "助動詞", "接続詞"]:
            yield node.surface.lower()
        node = node.next

def get_words(contents):
    words = [ ]
    for key, content in contents.items():
        tokens = [ ]
        for token in tokenize(content):
            tokens.append(token)
        words.append(tokens)
    return words

contents = get_contents() # すべてのファイルを読み込む
words = get_words(contents) # すべての単語を取得

from gensim import corpora, matutils
print(words[:10])
dictionary = corpora.Dictionary(words)
dictionary.filter_extremes()

[['特集', '/', 'journey', '刺激', '的', 'で', '優しい', 'アラブ', '国', '8', '/', '8', 'ドバイ', '誇る', '楽園', 'リゾート', '誘い', '旅', 'どこ', '国', '行く', 'ホテル', '泊まる', 'ほう', '重要', 'な', 'こと', 'ある', 'ドバイ', '旅', 'それ', 'だ', '首長', '強力', 'な', 'リーダーシップ', '中東', '随一', '観光', '国', 'なっ', 'た', 'ドバイ', 'ホテル', '数', '多い', '価格', '帯', 'タイプ', '様々', 'な', '自分', '求める', 'タイプ', 'ホテル', '明確', 'し', 'おか', 'ない', '結果', '不満足', 'な', '旅', 'なっ', 'しまう', 'こと', 'あり', '得る', '大きく', '分け', 'ドバイ', 'ホテル', '選び', 'パターン', '2', 'つ', '前回', '紹介', 'し', 'た', 'よう', 'な', '都市', '部', 'タワー', '型', 'ホテル', '宿泊', 'し', 'アクティブ', '行動', 'する', 'あるいは', '海', '面し', 'た', 'リゾート', 'ホテル', 'ブック', 'し', 'リラックス', 'し', 'た', '休日', '過ごす', 'どちら', 'なる', '個人', '的', '勧め', 'し', 'たい', 'の', '後者', '意外', '思う', 'しれ', 'ない', 'ドバイ', '海', '綺麗', 'だ', '中東', 'らしい', '澄ん', 'だ', '空', '深い', 'ブルー', '海', '織りなす', 'コントラスト', '見', 'いる', '異国', '情緒', '感じ', 'られる', 'ドバイ', '長年', 'ヨーロッパ', 'ツーリスト', 'たち', '人気', '集め', 'いる', '理由', 'リゾート', '気分', '満喫', 'できる', '質', '高い', 'ホテル', '数多く', 'ある', '他', 'なら', 'ない', 'ワン', '誇る', 'ザ・パーム',

In [ ]:
dictionary.token2id

{'"': 0,
 ',': 1,
 '-': 2,
 '/': 3,
 '0120': 4,
 '10': 5,
 '100': 6,
 '101': 7,
 '230': 8,
 '399': 9,
 '4': 10,
 '440': 11,
 '8': 12,
 '850': 13,
 '99': 14,
 '=': 15,
 'address': 16,
 'box': 17,
 'free': 18,
 'journey': 19,
 'km': 20,
 'livedoorhomme': 21,
 'one': 22,
 'only': 23,
 'photo': 24,
 'po': 25,
 'stay': 26,
 'tel': 27,
 'text': 28,
 'the': 29,
 'uae': 30,
 'あり': 31,
 'あるいは': 32,
 'いい': 33,
 'いただける': 34,
 'おか': 35,
 'き': 36,
 'く': 37,
 'くれる': 38,
 'ここ': 39,
 'こちら': 40,
 'させる': 41,
 'しまう': 42,
 'しれ': 43,
 'それ': 44,
 'たい': 45,
 'たち': 46,
 'ため': 47,
 'だっ': 48,
 'だろ': 49,
 'つ': 50,
 'できる': 51,
 'とおり': 52,
 'どこ': 53,
 'どちら': 54,
 'なく': 55,
 'なら': 56,
 'ならでは': 57,
 'なれ': 58,
 'ひとつ': 59,
 'ほう': 60,
 'み': 61,
 'もっ': 62,
 'もの': 63,
 'ら': 64,
 'らしい': 65,
 'られ': 66,
 'られる': 67,
 'アクティブ': 68,
 'アラブ': 69,
 'クラブ': 70,
 'ゲストハウス': 71,
 'コントラスト': 72,
 'ゴージャス': 73,
 'スパ': 74,
 'センス': 75,
 'タイプ': 76,
 'タワー': 77,
 'ディナー': 78,
 'ドバイ': 79,
 'ハイ': 80,
 'ハウス': 81,
 'ハワイ': 82,
 'バイ': 83,
 'バー': 84,
 

In [ ]:
len(dictionary.token2id)

22910

In [ ]:
def get_id_vector(dictionary, content): # 文章を単語IDのリストにする
    tokens = [ ]
    for token in tokenize(content):
        tokens.append(token)
    return dictionary.doc2idx(tokens)

def get_class_id(filename): # ファイル名からカテゴリIDを取得する
    dirlist = get_dir_list()
    for i, dirname in enumerate(dirlist):
        if dirname in filename:
            return i
    return -1

X = [ ]
y = [ ]
for filename, content in contents.items():
    #動作確認用
    #print(filename)
    #print("x=", get_id_vector(dictionary, content))
    #print("y=", get_class_id(filename))
    #break
    X.append(get_id_vector(dictionary, content))
    y.append(get_class_id(filename))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.utils import np_utils, to_categorical
from keras.preprocessing import sequence
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

maxlen = 200
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print(X_train[0])

y_train = to_categorical(y_train, 9)
y_test = to_categorical(y_test, 9)
print(y_train[0])

[ 5798  7584  9403    -1   788  7433  8256   794  2885    -1    -1   487
  1108   862 12300    -1  1348  6772   274    -1  1392    -1   275 12202
  2118    -1  3926    -1   604  1618    -1  8256    -1  4999    -1    -1
  7472  1977    -1    -1   582  1124   434  2885    51    63  1635    -1
   328   434  3440  3166  1395    -1  1108  2885    -1    -1    51   862
 12300    -1  1348  6772  2885   415    -1  1108   148   360  9102    31
  5568  3166  1395   434  5592  3204   415    -1    -1  2492  1108  9403
   739    -1    -1   588  6171   622   739    -1    -1    -1    -1  3159
  5704  2052    -1    -1   345   794  1392    -1    -1    -1  6079  1475
  1527   727  2726    67  1108    31  9857  1124  2885    -1    -1  1123
   169    -1   344   338    -1   200    -1   794   118   304   140  5052
     1   306   400     5   277    -1   140  1989     1   306   400    -1
  1590    -1   862 12300  6772  1348  6772  1108  3926   852   817 14290
 15208 15210  3263  5212  6192 14274 15209   624  3

LSTMかGRUを使って、 `X_train` と `y_train` で学習してみる

- パラメータ調整
- LSTMを多層にする
- bi-directional LSTM にする
- 任意の文章を入力して判定できるように
- https://bit.ly/2G1XMvA

- 20 News Groups データセットも試す

過学習しちゃうとき：
- model.summary()で見たときのパラメータ数が多すぎる
- Denseの層があるならDropoutを使う(出力層以外で)
- Embeddingの埋め込み次元(output_dim)を減らす
- EarlyStoppingを使う
- 前処理(tokenize()等)をがんばる

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=len(dictionary.token2id)+2,
                        output_dim=256, input_length=maxlen))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(9, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train+2, y_train, batch_size=256,
          epochs=10, validation_split=0.1)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 200, 256)          5865472   
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                82176     
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 585       
Total params: 5,948,233
Trainable params: 5,948,233
Non-trainable params: 0
_________________________________________________________________
Train on 5304 samples, validate on 590 samples
Epoch 1/10
5304/5304 [==============================] - 28s 5ms/step - loss: 2.0769 - acc: 0.4644 - val_loss: 1.8153 - val_acc: 0.3949
Epoch 2/10
5304/5304 [==============================] - 27s 5ms/step - loss: 1.5453 - acc: 0.5715 - val_loss: 1.4828 - val_acc: 0.4915
Epoch 3/10
5304/5304 [==============================] - 2

In [ ]:
test_str = "サンコーは、電動で車のジャッキアップが可能なパンタグラフ式の「シガーソケット電動ジャッキ」を発売した。価格は7,980円（税込）。スタッドレスなどの冬タイヤへの交換、積雪時のチェーン装着などで役に立つだろう。"
test_vector = [get_id_vector(dictionary, test_str)]
test_vector = sequence.pad_sequences(test_vector,maxlen=maxlen)
results = model.predict(test_vector+2)
print(results)
categories = get_dir_list()
print(categories[results.argmax()])

[[0.06665968 0.70043707 0.03291676 0.0242016  0.05330076 0.00629896
  0.00182113 0.01904454 0.09531938]]
kaden-channel
